In [1]:
import os
import joblib
import numpy as np
import pandas as pd
from scipy.sparse import hstack, vstack
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.utils import resample
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [2]:
diretorio_destino = '../vetores'

tfidf_title = joblib.load(f"{diretorio_destino}/tfidf_title.pkl")
tfidf_abstract = joblib.load(f"{diretorio_destino}/tfidf_abstract.pkl")
tfidf_combined = hstack([tfidf_title, tfidf_abstract])

In [3]:
eps = 0.5
min_samples = 10
batch_size = 20000

In [4]:
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine', n_jobs=-1)

In [5]:
all_labels = []

for i in range(0, tfidf_combined.shape[0], batch_size):
    batch = tfidf_combined[i:i+batch_size]
    print(f"Processando batch {i // batch_size + 1}: {batch.shape[0]} itens.")
    batch_labels = dbscan.fit_predict(batch)
    all_labels.extend(batch_labels)

Processando batch 1: 20000 itens.
Processando batch 2: 20000 itens.
Processando batch 3: 20000 itens.
Processando batch 4: 20000 itens.
Processando batch 5: 20000 itens.
Processando batch 6: 20000 itens.
Processando batch 7: 20000 itens.
Processando batch 8: 20000 itens.
Processando batch 9: 20000 itens.
Processando batch 10: 20000 itens.
Processando batch 11: 20000 itens.
Processando batch 12: 20000 itens.
Processando batch 13: 20000 itens.
Processando batch 14: 20000 itens.
Processando batch 15: 20000 itens.
Processando batch 16: 20000 itens.
Processando batch 17: 20000 itens.
Processando batch 18: 20000 itens.
Processando batch 19: 20000 itens.
Processando batch 20: 20000 itens.
Processando batch 21: 20000 itens.
Processando batch 22: 20000 itens.
Processando batch 23: 20000 itens.
Processando batch 24: 20000 itens.
Processando batch 25: 20000 itens.
Processando batch 26: 20000 itens.
Processando batch 27: 20000 itens.
Processando batch 28: 20000 itens.
Processando batch 29: 20000 i

In [6]:
all_labels = np.array(all_labels)
print(f"Clusterização concluída. Total de clusters encontrados: {len(set(all_labels)) - (1 if -1 in all_labels else 0)}")

Clusterização concluída. Total de clusters encontrados: 6


In [7]:
n_samples = 10000
X_sample, labels_sample = resample(tfidf_combined, all_labels, n_samples=n_samples, random_state=42)

In [8]:
if len(set(labels_sample)) > 1:
    silhouette_avg = silhouette_score(X_sample, labels_sample)
    print(f"Silhouette Score (subamostra): {silhouette_avg}")
else:
    print("DBSCAN não formou clusters significativos na subamostra.")

Silhouette Score (subamostra): -0.00023132931628465044


In [ ]:
# pca = PCA(n_components=2, random_state=42)
# X_reduced = pca.fit_transform(X_sample.toarray())

In [ ]:
# plt.figure(figsize=(10, 6))
# unique_labels = set(labels_sample)
# for label in unique_labels:
#     mask = labels_sample == label
#     plt.scatter(X_reduced[mask, 0], X_reduced[mask, 1], label=f'Cluster {label}' if label != -1 else "Ruído", s=10)
# plt.title("Dispersão com PCA (DBSCAN)")
# plt.legend()
# plt.show()

In [ ]:
# df = pd.read_parquet("../data/dblp-v10-processado.parquet")
# df['cluster_dbscan'] = all_labels

In [ ]:
# for cluster in unique_labels:
#     if cluster == -1:  # Ignorar ruído
#         continue
#     cluster_text = df[df['cluster_dbscan'] == cluster]['title'].apply(lambda x: ' '.join(x)).str.cat(sep=' ')
#     wordcloud = WordCloud(width=800, height=400, background_color='white').generate(cluster_text)
#     plt.figure(figsize=(8, 4))
#     plt.imshow(wordcloud, interpolation='bilinear')
#     plt.axis('off')
#     plt.title(f'Nuvem de Palavras - Cluster {cluster}')
#     plt.show()